<a href="https://colab.research.google.com/github/Farynus13/Computer-VIsion-Training/blob/main/car_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!nvidia-smi
!pip install scikit-learn==1.4.0


Sat Mar 16 12:15:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow import keras
import tensorflow as tf
import cv2
from sklearn.cluster import KMeans
import time

In [ ]:
path="/content/data/"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import zipfile
zip_name = '/content/gdrive/My Drive/segmentation_data.zip'
zip_ref = zipfile.ZipFile(zip_name, 'r')
zip_ref.extractall(path)
zip_ref.close()

Mounted at /content/gdrive/


In [ ]:
import glob

data = glob.glob(path + "imgs/*.jpg", recursive=True)
labels = glob.glob(path + "masks/*.jpg", recursive=True)
train_examples = [(data[i], labels[i]) for i in range(len(data))]
print(len(data))

9888


In [ ]:
colors = []
count = 0
for mask in labels:
	seg = cv2.imread(mask)
	count+=1
	if count == 500:
		break
	colors.append(seg.reshape(seg.shape[0]*seg.shape[1], 3))
colors = np.array(colors)
colors = colors.reshape((colors.shape[0]*colors.shape[1],3))

km = KMeans(5)
km.fit(colors)

KMeans(n_clusters=5)

In [ ]:
class DataGenerator(tf.compat.v2.keras.utils.Sequence):
	def __init__(self, list_examples, batch_size=64, dim=(244,244,3),n_classes=5):
		# Constructor of the data generator.
		self.dim = dim
		self.batch_size = batch_size
		self.list_examples = list_examples
		self.n_classes = n_classes
		self.on_epoch_end()

	def __len__(self):
		# Denotes the number of batches per epoch
		return int(np.floor(len(self.list_examples) / self.batch_size))

	def __getitem__(self, index):
		# Generate one batch of data
		indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

		# Find list of IDs
		list_IDs_temp = [self.list_examples[k] for k in indexes]

		# Generate data
		X, y = self.__data_generation(list_IDs_temp)

		return X, y

	def on_epoch_end(self):
		# This function is called at the end of each epoch.
		self.indexes = np.arange(len(self.list_examples))

	def __data_generation(self, list_IDs_temp):
		# Initialization
		X = []
		y = []

		# Generate data
		for i, ID in enumerate(list_IDs_temp):
			# Store sample
			img = cv2.imread(ID[0])
			img = cv2.equalizeHist(img)
			img = img / 255
			X.append(img)
			# cv2.imshow('img', img)
			# cv2.waitKey(0)
			mask = cv2.imread(ID[1])
			mask = self.ColorsToClass(mask)
			# y.append(mask)
			# for i in range(5):
			# 	cv2.imshow('mask', mask[:,:,i])
			# 	cv2.waitKey(0)
			y.append(mask)
		X = np.array(X)
		y = np.array(y)

		return X,y

	def ColorsToClass(self,seg):
		s = seg.reshape((seg.shape[0]*seg.shape[1],3))
		s = km.predict(s)
		s = s.reshape((seg.shape[0], seg.shape[1]))
		return s

		n = len(km.cluster_centers_)

		cls = np.zeros((seg.shape[0], seg.shape[1], n))

		for i in range(n):
			m = np.copy(s)
			m[m!=i] = 0
			m[m!=0] = 1

			cls[:,:,i]=m

		return cls

In [ ]:
# Parameters
width, height = 224, 224
batch_size = 64
n_classes = 5
params = {'dim': (height, width, 3),
          'batch_size': batch_size,
          'n_classes': n_classes
}
# Define the generators
training_generator = DataGenerator(train_examples, **params)

In [ ]:

base_model = keras.applications.vgg16.VGG16(
    include_top=False, input_shape=(width, height, 3))

layer_names = [
    'block1_pool',
    'block2_pool',
    'block3_pool',
    'block4_pool',
    'block5_pool',
]
base_model_outputs = [base_model.get_layer(
    name).output for name in layer_names]
base_model.trainable = False

VGG_16 = tf.keras.models.Model(base_model.input,
                               base_model_outputs)

def fcn8_decoder(convs, n_classes):
	f1, f2, f3, f4, p5 = convs

	n = 4096
	c6 = tf.keras.layers.Conv2D(
		n, (7, 7), activation='relu', padding='same',
	name="conv6")(p5)
	c7 = tf.keras.layers.Conv2D(
		n, (1, 1), activation='relu', padding='same',
	name="conv7")(c6)

	f5 = c7

	# upsample the output of the encoder
	# then crop extra pixels that were introduced
	o = tf.keras.layers.Conv2DTranspose(n_classes, kernel_size=(
		4, 4), strides=(2, 2), use_bias=False)(f5)
	o = tf.keras.layers.Cropping2D(cropping=(1, 1))(o)

	# load the pool 4 prediction and do a 1x1
	# convolution to reshape it to the same shape of `o` above
	o2 = f4
	o2 = (tf.keras.layers.Conv2D(n_classes, (1, 1),
								activation='relu',
								padding='same'))(o2)

	# add the results of the upsampling and pool 4 prediction
	o = tf.keras.layers.Add()([o, o2])

	# upsample the resulting tensor of the operation you just did
	o = (tf.keras.layers.Conv2DTranspose(
		n_classes, kernel_size=(4, 4), strides=(2, 2),
	use_bias=False))(o)
	o = tf.keras.layers.Cropping2D(cropping=(1, 1))(o)

	# load the pool 3 prediction and do a 1x1
	# convolution to reshape it to the same shape of `o` above
	o2 = f3
	o2 = (tf.keras.layers.Conv2D(n_classes, (1, 1),
								activation='relu',
								padding='same'))(o2)

	# add the results of the upsampling and pool 3 prediction
	o = tf.keras.layers.Add()([o, o2])

	# upsample up to the size of the original image
	o = tf.keras.layers.Conv2DTranspose(
		n_classes, kernel_size=(8, 8), strides=(8, 8),
	use_bias=False)(o)

	# append a softmax to get the class probabilities
	o = tf.keras.layers.Activation('softmax')(o)
	return o

def segmentation_model():

	inputs = keras.layers.Input(shape=(width, height, 3))
	convs = VGG_16(inputs)
	outputs = fcn8_decoder(convs, 5)
	model = tf.keras.Model(inputs=inputs, outputs=outputs)

	return model


opt = keras.optimizers.Adam(learning_rate=0.001)


model = segmentation_model()
model.compile(optimizer=opt,
			loss=tf.keras.losses.SparseCategoricalCrossentropy(
				from_logits=True),
			metrics=['accuracy'])


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
EPOCHS = 20

model_history = model.fit(training_generator, epochs=EPOCHS, verbose=1,batch_size=batch_size)

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/histogram.cpp:3440: error: (-215:Assertion failed) _src.type() == CV_8UC1 in function 'equalizeHist'


In [ ]:
model.save('/content/seg_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import files
files.download('seg_model.h5')